In [17]:
import pandas as pd
import mysql.connector
pd.options.display.max_columns = None

In [18]:
df = pd.read_csv('attacks_clima.csv', index_col=0)
df.head(2)

,case_number,year,type,country,area,location,activity,name,age,time,species_,injury,fecha_limpia,fatal,sex,pais,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,wind_950mb,wind_900mb,wind_850mb,wind_800mb,wind_750mb,wind_700mb,wind_650mb,wind_600mb,wind_550mb,wind_500mb,wind_450mb,wind_400mb,wind_350mb,wind_300mb,wind_250mb,wind_200mb,index,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.direction,wind10m.speed
0,2018.06.25,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,57,18h00,White shark,"No injury to occupant, outrigger canoe and pad...",Jun,N,F,USA,4.953125,6.15625,6.8125,6.296875,5.03125,5.296875,5.234375,5.09375,5.078125,6.015625,6.5,6.84375,7.296875,5.265625,4.015625,-0.3125,3.125,3.375,3.6875,3.75,3.609375,3.625,4.046875,4.5625,5.140625,5.671875,6.1875,6.59375,7.0,7.546875,7.90625,7.21875,31.5,97.5,5.078125,-9999.0,-9999.0,-9999.0,11.421875,9.6875,4.25,1013.703125,1.234375,0.0,192.109375,2.921875
1,2018.06.18,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,AdysonÂ McNeely,11,14h00 -15h00,NaN,Minor injury to left thigh,Jun,N,F,USA,4.953125,6.15625,6.8125,6.296875,5.03125,5.296875,5.234375,5.09375,5.078125,6.015625,6.5,6.84375,7.296875,5.265625,4.015625,-0.3125,3.125,3.375,3.6875,3.75,3.609375,3.625,4.046875,4.5625,5.140625,5.671875,6.1875,6.59375,7.0,7.546875,7.90625,7.21875,31.5,97.5,5.078125,-9999.0,-9999.0,-9999.0,11.421875,9.6875,4.25,1013.703125,1.234375,0.0,192.109375,2.921875


In [19]:
class Cargar:
    
    def __init__(self, nombre_bbdd, contraseña):
       
        self.nombre_bbdd = nombre_bbdd
        self.contraseña = contraseña

 
    def crear_bbdd(self):

        mydb = mysql.connector.connect(host="localhost",
                                       user="root",
                                       password=f'{self.contraseña}') 
        mycursor = mydb.cursor()
        print("Conexión realizada con éxito")

        try:
            mycursor.execute(f"CREATE DATABASE IF NOT EXISTS {self.nombre_bbdd};")
            
        except:
            print("La BBDD ya existe")
            
   
    def crear_insertar_tabla(self, query):
        
        mydb = mysql.connector.connect(host="localhost",
                                       user="root",
                                       password=f'{self.contraseña}', 
                                       database=f"{self.nombre_bbdd}") 
        mycursor = mydb.cursor()
        
        try:
            mycursor.execute(query)
            mydb.commit()
          
        except mysql.connector.Error as err:
            print(err)
            print("Error Code:", err.errno)
            print("SQLSTATE", err.sqlstate)
            print("Message", err.msg)
            

    def check_ciudades(self):
    
        mydb = mysql.connector.connect(user='root',
                                      password=f"{self.contraseña}",
                                      host='127.0.0.1',
                                      database=f"{self.nombre_bbdd}")
        mycursor = mydb.cursor()

        
        query_existe_ciudad = f"""
                SELECT DISTINCT ciudad FROM localidades
                """
        mycursor.execute(query_existe_ciudad)
        ciudades = mycursor.fetchall()
        return ciudades
    
   
    def sacar_id_ciudad(self, ciudad):
        
        mydb = mysql.connector.connect(user='root',
                                       password= f'{self.contraseña}',
                                       host='127.0.0.1', 
                                       database=f"{self.nombre_bbdd}")
        mycursor = mydb.cursor()
        
        try:
            query_sacar_id = f"SELECT idlocalidad FROM localidades WHERE ciudad = '{ciudad}'"
            mycursor.execute(query_sacar_id)
            id_ = mycursor.fetchall()[0][0]
            return id_
        
        except: 
            return "Sorry, no tenemos esa ciudad en la BBDD y por lo tanto no te podemos dar su id. "
     
    
    def sacar_id_fecha(self, timepoint, fecha):
        mydb = mysql.connector.connect(user='root', password=f'{self.contraseña}',
                                          host='127.0.0.1', database=f"{self.nombre_bbdd}")
        mycursor = mydb.cursor()

        try:
            query_sacar_id = f"SELECT idtiempo FROM tiempo WHERE timepoint = {timepoint} and fecha = '{fecha}'"
            mycursor.execute(query_sacar_id)
            id_ = mycursor.fetchall()[0][0]
            return id_
        
        except: 
             return "Sorry, no tenemos esa fecha en la BBDD y por lo tanto no te podemos dar su id. "

In [20]:
tabla_ataques = """
CREATE TABLE IF NOT EXISTS `tiburones`.`ataques` (
  `case_number` INT NOT NULL AUTO_INCREMENT,
  `year` INT NOT NULL,
  `type` CHAR (100) NOT NULL,
  `country` VARCHAR (100) NOT NULL,
  `area` VARCHAR (100) NOT NULL,
  `location` VARCHAR (100) NOT NULL,
  `activity` VARCHAR (100) NOT NULL,
  `name` VARCHAR (100) NOT NULL,
  `age` VARCHAR (3) NOT NULL,
  `species_` VARCHAR (100) NOT NULL,
  `injury` VARCHAR (100) NOT NULL,
  `fecha_limpia` VARCHAR (100) NOT NULL,
  `fatal` VARCHAR (100) NOT NULL,
  `sex` VARCHAR(10) NOT NULL,
  `pais` VARCHAR (100) NOT NULL,
  PRIMARY KEY (`case_number`), )
ENGINE = InnoDB;
"""

In [21]:
tabla_clima = '''
CREATE TABLE IF NOT EXISTS `tiburones`.`clima` (
  `index` FLOAT NOT NULL AUTO_INCREMENT,
  `rh_950mb` FLOAT NOT NULL,
  `rh_900mb` FLOAT NOT NULL,
  `rh_850mb` FLOAT NOT NULL,
  `rh_800mb` FLOAT NOT NULL,
  `rh_750mb` FLOAT NOT NULL,
  `rh_700mb` FLOAT NOT NULL,
  `rh_650mb` FLOAT NOT NULL,
  `rh_600mb` FLOAT NOT NULL,
  `rh_550mb` FLOAT NOT NULL,
  `rh_500mb` DATETIME,
  `rh_450mb` FLOAT NOT NULL,
  `rh_400mb` FLOAT NOT NULL,
  `rh_350mb` FLOAT NOT NULL,
  `rh_300mb` FLOAT NOT NULL,
  `rh_250mb` FLOAT NOT NULL,
  `rh_200mb` FLOAT NOT NULL,
  `wind_950mb` FLOAT NOT NULL,
  `wind_900mb` FLOAT NOT NULL,
  `wind_850mb` FLOAT NOT NULL,
  `wind_800mb` FLOAT NOT NULL,
  `wind_750mb` FLOAT NOT NULL,
  `wind_700mb` FLOAT NOT NULL,
  `wind_650mb` FLOAT NOT NULL,
  `wind_600mb` FLOAT NOT NULL,
  `wind_550mb` FLOAT NOT NULL,
  `wind_500mb` FLOAT NOT NULL,
  `wind_450mb` FLOAT NOT NULL,
  `wind_400mb` FLOAT NOT NULL,
  `wind_350mb` FLOAT NOT NULL,
  `wind_300mb` FLOAT NOT NULL,
  `wind_250mb` FLOAT NOT NULL,
  `wind_2000mb` FLOAT NOT NULL,
  `timepoint` FLOAT NOT NULL, 
  `cloudcover` FLOAT NOT NULL, 
  `highcloud` FLOAT NOT NULL, 
  `midcloud` FLOAT NOT NULL, 
  `lowcloud` FLOAT NOT NULL, 
  `temp2m` FLOAT NOT NULL, 
  `lifted_index` FLOAT NOT NULL, 
  `rh2m` FLOAT NOT NULL, 
  `msl_pressure` FLOAT NOT NULL, 
  `prec_amount` FLOAT NOT NULL, 
  `snow_depth` FLOAT NOT NULL,
  `wind10m.direction` FLOAT NOT NULL,
  `wind10m.speed` FLOAT NOT NULL,

  
  PRIMARY KEY (`index`))
ENGINE = InnoDB;
'''

In [22]:
carga = Cargar("ataques_y_clima", "AlumnaAdalab")

In [23]:
carga

In [25]:
carga.crear_bbdd()

NotSupportedError: Authentication plugin 'caching_sha2_password' is not supported

In [ ]:
carga.crear_insertar_tabla(tabla_clima)
carga.crear_insertar_tabla(tabla_ataques)

No entendemos el error que nos sale al llamar a la función de crear la bbdd. Igualmente dejamos la creacion de la bbdd y de las tablas planteado.